In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt



c:\Users\Alessandro\anaconda3\envs\tfgpu\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_data = pd.read_csv('mnist_train_small.csv', header=None)
test_data = pd.read_csv('mnist_test.csv', header=None)


In [3]:
train_data

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19998,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_data.values

array([[6, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       [7, 0, 0, ..., 0, 0, 0],
       ...,
       [2, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [5]:
def train_test(data,splitting_ratio=0.05):
    '''splits and normalizes data'''
    data = np.array(data)
    m, n = data.shape
    print('m (# of examples) = ', m , '\nn = (# of features + # of labels)', n)
    np.random.shuffle(data) # shuffle before splitting into dev and training sets this function only shuffles along the first axis (rows, i.e. it shuffles examples in our case)
    number_test=int(m*splitting_ratio)
    print('test data size: ',number_test)
    data_dev = data[0:number_test,:] 
    Y_dev = data_dev[:,0] #first COLUMN is features
    X_dev = data_dev[:,1:n]
    X_dev = X_dev / 255.

    data_train = data[number_test:m]
    Y_train = data_train[:,0]
    X_train = data_train[:,1:n]
    X_train = X_train / 255.
    n_t,m_t = X_train.shape
    print( 'Data prepared.\n# of examples for training: ', n_t)
    return X_train, Y_train, X_dev, Y_dev

In [6]:
x_train, y_train, x_val, y_val = train_test(train_data)

m (# of examples) =  20000 
n = (# of features + # of labels) 785
test data size:  1000
Data prepared.
# of examples for training:  19000


In [7]:
tf_data_train= tf.data.Dataset.from_tensor_slices((x_train,y_train))
tf_data_val= tf.data.Dataset.from_tensor_slices((x_val,y_val))

In [8]:
tf_data_train

<TensorSliceDataset element_spec=(TensorSpec(shape=(784,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

now I build a NN with tensorflow that does the same. the first layer has 784 input nodes, then I add a middle layer with a given number of nodes and a relu actiavtion function, and a final outpout layer


so the pipeline is

csv->pandas->numpy->tensorflow feature/label

In [9]:
x_train.shape

(19000, 784)

In [10]:
def create_model(lr=0.1,n_nodes=10):
    '''creates and compile the model
    model takes as input the learnign rate and the feature layer (defined elsewhere)'''
    model = tf.keras.models.Sequential() #general type of the model
    #model.add(my_feature_layer)#layer containing the feature columns
    model.add(tf.keras.layers.Dense(input_shape=(784,), units=n_nodes, activation='relu', name='Hidden1')) #hidden layer
    #model.add(tf.keras.layers.Dropout(rate=.2))
    #model.add(tf.keras.layers.Dense( units=n_nodes, activation='relu', name='Hidden2')) #hidden layer
    model.add(tf.keras.layers.Dense(units=10,activation='softmax', name='Output')) #output layer, must have 1 node ofr each class (in this case 10)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),loss='categorical_crossentropy', metrics=['accuracy']) #check what metric(s) to use! (accuracy)
    return model


TODO:

1) check which metrics/activation function to use 

2) prepare/normalize data, split in train/test, etc

3) feed data into NN

4) set labels to categorical

5) train the model

6) Obtain metrics

7) make predictions

In [11]:
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)

# Set data types for float features
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')

In [12]:
x_train.shape

(19000, 784)

In [13]:
model =create_model(0.01,100)


In [14]:
history = model.fit(x_train, y_train, epochs=20, batch_size=20, validation_data=(x_val, y_val))

Epoch 1/20
950/950 [==============================] - 5s 4ms/step - loss: 0.3361 - accuracy: 0.8981 - val_loss: 0.1960 - val_accuracy: 0.9420
Epoch 2/20
950/950 [==============================] - 4s 4ms/step - loss: 0.1994 - accuracy: 0.9425 - val_loss: 0.1664 - val_accuracy: 0.9490
Epoch 3/20
950/950 [==============================] - 4s 4ms/step - loss: 0.1717 - accuracy: 0.9507 - val_loss: 0.2073 - val_accuracy: 0.9550
Epoch 4/20
950/950 [==============================] - 5s 6ms/step - loss: 0.1501 - accuracy: 0.9578 - val_loss: 0.1729 - val_accuracy: 0.9540
Epoch 5/20
950/950 [==============================] - 5s 5ms/step - loss: 0.1391 - accuracy: 0.9614 - val_loss: 0.2222 - val_accuracy: 0.9460
Epoch 6/20
950/950 [==============================] - 6s 7ms/step - loss: 0.1382 - accuracy: 0.9618 - val_loss: 0.1985 - val_accuracy: 0.9480
Epoch 7/20
950/950 [==============================] - 4s 4ms/step - loss: 0.1302 - accuracy: 0.9659 - val_loss: 0.2790 - val_accuracy: 0.9530
Epoch 

In [15]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden1 (Dense)             (None, 100)               78500     
                                                                 
 Output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
model.count_params()

79510